# Arrays

In [ ]:
#include <iostream>
using std::cout, std::endl;

In [ ]:
/*
    This method prints the address of the data
    i.e. where in memory does the "thing" reside
*/
template <class T>
void where(T const& thing) {
    printf("where: %p\n", &thing);
}

In [ ]:
/*
  This method prints the bytes in RAM that a given variable has
  i.e. what are the actual 1s and 0s that occupy the space given to "thing"
  Prints the bytes in hexidecimal.
*/
template <class T>
void bytes(T const& thing) {
    unsigned char* addr = (unsigned char*)&thing;
    printf("bytes: 0x");
    for (int i = sizeof(T) - 1; i >= 0; i--) {
        printf("%02x", addr[i]);
    }
    printf("\n");
}

In [38]:
int foo = 7;
int bar = 4;

In [39]:
where(foo);
where(bar);

where: 0x40091ce0b0
where: 0x40091ce0b4


In [40]:
bytes(foo);

bytes: 0x00000007


In [41]:
int* fooptr = &foo;
where(foo);
bytes(fooptr);

where: 0x40091ce0b0
bytes: 0x00000040091ce0b0


In [42]:
bytes(*fooptr);

bytes: 0x00000007


In [43]:
bytes(*(fooptr + 1))

bytes: 0x00000004


In [44]:
bytes(fooptr);
bytes(*fooptr);

fooptr++;  cout << "fooptr++" << endl;

bytes(fooptr);
bytes(*fooptr);

bytes: 0x00000040091ce0b0
bytes: 0x00000007
fooptr++
bytes: 0x00000040091ce0b4
bytes: 0x00000004


In [46]:
long long big_foo = 3;
long long big_bar = 5;

In [47]:
where(big_foo);
where(big_bar);

where: 0x40091ce0c0
where: 0x40091ce0c8


In [48]:
long long *big_foo_ptr = &big_foo;

bytes(big_foo_ptr);
bytes(*big_foo_ptr);

big_foo_ptr++;  cout << "big_foo_ptr++" << endl;

bytes(big_foo_ptr);
bytes(*big_foo_ptr);

bytes: 0x00000040091ce0c0
bytes: 0x0000000000000003
big_foo_ptr++
bytes: 0x00000040091ce0c8
bytes: 0x0000000000000005


When you "add one" to a pointer, you get a new memory address that is one type-width away from the first.

So adding one to an `int*` moves 4 bytes.

Adding one to a `long long*` moves 8 bytes.

In [49]:
int foo = 7;
int* fooptr = &foo;

where(*fooptr);
where(*(fooptr + 1));

where: 0x40091ce0d8
where: 0x40091ce0dc


In [50]:
where(fooptr[0]);
where(fooptr[1]);

where: 0x40091ce0d8
where: 0x40091ce0dc


In [61]:
where(fooptr[1]);

/* is equivalent to */

where(*(fooptr + 1));

where: 0x400000000b
where: 0x400000000b


if
```c++
int* foo;
```
then

```c++
foo[1]
```
is the same as saying:

> move 1 `int`s' worth of bytes from `foo` and get the value—i.e. `*(foo + 1)`

So, if you new you had a sequence of values all of the same type, all right next to each other in memory, you could easily get to all of them if you new the address of the first one.

<div style='font-size:200pt'>🤔</div>

## Array Allocation

In [62]:
int* quux = new int[10];

In [63]:
where(*quux)

where: 0x40282401e0


In [64]:
where(*(quux + 1))

where: 0x40282401e4


In [65]:
where(quux[0])

where: 0x40282401e0


In [66]:
where(quux[1])

where: 0x40282401e4


In [67]:
where(quux[2])

where: 0x40282401e8


- `new` array syntax
- `[]` on arrays does pointer math

In [68]:
bytes(quux[0])

bytes: 0x2c21d5c0


In [69]:
for (int i = 0; i < 10; i++) {
    bytes(quux[i]);
}

bytes: 0x2c21d5c0
bytes: 0x00000040
bytes: 0x00000000
bytes: 0x00000000
bytes: 0x171e8300
bytes: 0x00000040
bytes: 0x28235780
bytes: 0x00000040
bytes: 0x9b4b1000
bytes: 0x0000ffff


- Calling `new T[n]` gives you space for `n` `T`s, but doesn't create any `T`s yet.
- There are bytes there, but they are just weeds growing in the empty lot you just purchased

In [70]:
for (int i = 0; i < 10; i++) {
    quux[i] = i;
}

In [71]:
for (int i = 0; i < 10; i++) {
    bytes(quux[i]);
}

bytes: 0x00000000
bytes: 0x00000001
bytes: 0x00000002
bytes: 0x00000003
bytes: 0x00000004
bytes: 0x00000005
bytes: 0x00000006
bytes: 0x00000007
bytes: 0x00000008
bytes: 0x00000009


In [72]:
for (int i = 10; i < 20; i++) {
    bytes(quux[i]);
}

bytes: 0x00000041
bytes: 0x00000000
bytes: 0x00000026
bytes: 0x00000000
bytes: 0x9b4aa000
bytes: 0x0000ffff
bytes: 0x314e5a5f
bytes: 0x635f5f32
bytes: 0x676e696c
bytes: 0x36354e5f


Remember, the `[ ]` operator just does some pointer math and jumps to the specified slot in memory.

It doesn't know whether that slot in memory is available, has useful data, etc. 

How can you use an array to store items?

- preallocate an array that is "big enough"
- keep track of how many items you have
- add new items to the next available position in the array

**Issues**
- what happens with "big enough" isn't big enough anymore (i.e. you run out of room)?
  - do you stop adding items? (this is bad)
  - do you run off the end of the array? (this is REALLY bad)
- what happens if you loose track of how many items you have?


### Pro-tip 
> The goal is not to write code that you can get to work  
> The goal is to write code that you **can't get wrong**

## Vector

- Wrap an array
- Keep track of how big the array is and how many items we've added
- If we run out of room, we get a bigger array, copy the items over, and proceed
- always add the next item to the correct spot

### `build_vector.cpp`

Write and discuss:

- Constructor and destructor
- `push_back` + `grow`
- `pop_back`
- `size`
- `operator []`

Note:
- `pop_back` just decrements size. Nothing is done to the memory.
- what is the significance of returning by value from `operator[]`?

## Key Ideas
- Arrays
  - contiguous memory allocation
- Vectors